In [97]:
import sys
from typing import Iterable
import warnings

import nibabel
from neurolang import frontend as fe
import numpy as np
import pandas as pd
import nilearn.datasets
import nilearn.image
import nilearn.plotting


import xml.etree.ElementTree as ET
import json
from neurolang.frontend import NeurolangPDL, ExplicitVBR, ExplicitVBROverlay
from rdflib import RDFS
from sklearn.model_selection import KFold

from typing import Callable, Iterable

In [98]:
iobc = nilearn.datasets.utils._fetch_files(
    nilearn.datasets.utils._get_dataset_dir('IOBC'),
    [
        (
            'iobc.xrdf',
            'http://data.bioontology.org/ontologies/IOBC/download?'
            'apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb&download_format=rdf',
            {'move': 'iobc.xrdf'}
        )
    ]
)[0]

In [99]:
nl = NeurolangPDL()
nl.load_ontology(iobc)

In [100]:
# NeuroSynth
mni_mask = nilearn.image.resample_img(
    nibabel.load(nilearn.datasets.fetch_icbm152_2009()["gm"]),
    np.eye(3) * 2
)

ns_database_fn, ns_features_fn = nilearn.datasets.utils._fetch_files(
    "neurolang",
    [
        (
            "database.txt",
            "https://github.com/neurosynth/neurosynth-data"
            "/raw/master/current_data.tar.gz",
            {"uncompress": True},
        ),
        (
            "features.txt",
            "https://github.com/neurosynth/neurosynth-data"
            "/raw/master/current_data.tar.gz",
            {"uncompress": True},
        ),
    ],
)

ns_database = pd.read_csv(ns_database_fn, sep="\t")
ns_database = ns_database[["x", "y", "z", "id"]]

ns_features = pd.read_csv(ns_features_fn, sep="\t")
ns_docs = ns_features[["pmid"]].drop_duplicates()
ns_terms = pd.melt(
    ns_features, var_name="term", id_vars="pmid", value_name="TfIdf"
).query("TfIdf > 1e-3")[["term", "pmid"]]

In [101]:
SelectedStudy = nl.add_uniform_probabilistic_choice_over_set(
    ns_docs, name="SelectedStudy"
)

In [102]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

ns_doc_folds = pd.concat(
    ns_docs.iloc[train].assign(fold=[i] * len(train))
    for i, (train, _) in enumerate(kfold.split(ns_docs))
)


StudyFolds = nl.add_tuple_set(ns_doc_folds, name='StudyFolds')

TermInStudy = nl.add_tuple_set(ns_terms, name="TermInStudy")
FocusReported = nl.add_tuple_set(ns_database, name="FocusReported")
Voxel = nl.add_tuple_set(
    nibabel.affines.apply_affine(
        mni_mask.affine,
        np.transpose(mni_mask.get_fdata().nonzero())
    ),
    name='Voxel'
)

terms_det = nl.add_tuple_set(
        ns_terms.term.unique(), name='terms_det'
)

study_det = nl.add_tuple_set(
        ns_docs.pmid.unique(), study_det='terms_det'
) 

label = nl.new_symbol(name=str(RDFS.label))
related = nl.new_symbol(name='http://www.w3.org/2004/02/skos/core#related')
altLabel = nl.new_symbol(name='http://www.w3.org/2004/02/skos/core#altLabel')
subclass_of = nl.new_symbol(name=str(RDFS.subClassOf))

TypeError: add_tuple_set() got an unexpected keyword argument 'study_det'

In [ ]:
@nl.add_symbol
def first_word(name: str) -> str:
    return name.split(" ")[0]

@nl.add_symbol
def word_lower(name: str) -> str:
    return name.lower()

@nl.add_symbol
def mean(iterable: Iterable) -> float:
    return np.mean(iterable)


@nl.add_symbol
def std(iterable: Iterable) -> float:
    return np.std(iterable)

@nl.add_symbol
def agg_create_region_overlay_MNI(
    x: Iterable, y: Iterable, z: Iterable, p: Iterable
) -> ExplicitVBR:
    voxels = nibabel.affines.apply_affine(
        np.linalg.inv(mni_mask.affine),
        np.c_[x, y, z]
    )
    return ExplicitVBROverlay(
        voxels, mni_mask.affine, p, image_dim=mni_mask.shape
    )

nl.add_symbol(np.exp, name="exp", type_=Callable[[float], float])

In [ ]:
with nl.scope as e:
    e.ontology_related[e.ne, e.l] = (
        label(e.e, e.ne) &
        related(e.e, e.r) &
        label(e.r, e.nr) &
        (e.l == word_lower[e.nr])
    )
    
    e.ontology_synonym[e.ne, e.l] = (
        label(e.e, e.ne) &
        altLabel(e.e, e.r) &
        (e.l == word_lower[e.r])
    )
    
    e.res[e.entity, e.relation, e.term] = (
        e.ontology_related[e.entity, e.term] &
        e.terms_det[e.entity] &
        e.terms_det[e.term] &
        (e.relation == 'related')
    )
    
    e.res[e.entity, e.relation, e.term] = (
        e.ontology_synonym[e.entity, e.term] &
        e.terms_det[e.entity] &
        e.terms_det[e.term] &
        (e.relation == 'synonym')
    )
    
    res = nl.solve_all()
    

In [ ]:
result = res['res'].as_pandas_dataframe()

In [95]:
result.entity.unique()

array(['genotype', 'treatment', 'therapy', 'medication', 'rest', 'neuron',
       'preparation', 'placebo', 'animal', 'facial expression', 'body',
       'speech', 'sound', 'voice', 'speech production', 'drugs', 'drug',
       'stroke', 'access', 'epilepsy', 'face recognition', 'touch',
       'hyperactivity disorder', 'youth', 'childhood', 'children', 'tone',
       'diffusion tensor', 'latency', 'early stage', 'aging',
       'development', 'growing', 'age', 'human', 'pet', 'face',
       'features', 'emotion', 'food', 'addiction', 'skin', 'anatomy',
       'pathophysiology', 'illusion', 'cognitive function',
       'cognitive performance', 'view', 'visual information', 'vision',
       'visual', 'visual field', 'recognition', 'visual motion',
       'modality', 'contrast', 'visual stimulus', 'visual stimuli',
       'scene', 'midbrain', 'hearing', 'symptom', 'symptoms', 'pathology',
       'illness', 'disease', 'skin conductance', 'pain', 'sex', 'gender',
       'motor function', 't

In [11]:
related = result[result.relation == 'related']
related = related[related['entity'] != related['term']]

In [13]:
synonym = result[result.relation == 'synonym']
synonym = synonym[synonym['entity'] != synonym['term']]

In [15]:
with nl.environment as e:
    (e.VoxelReported @ e.exp(-e.d / 5.0))[e.x1, e.y1, e.z1, e.s] = (
        e.FocusReported(e.x2, e.y2, e.z2, e.s)
        & e.Voxel(e.x1, e.y1, e.z1)
        & (e.d == e.EUCLIDEAN(e.x1, e.y1, e.z1, e.x2, e.y2, e.z2))
        & (e.d < 4)
    )
    
    nl.solve_all()

/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

In [93]:
hash_list = {}
for entity, term in synonym[['entity', 'term']].values:
    
    cut = synonym[synonym.entity == entity].term.values
    all_terms = np.append(cut, entity)
    all_terms.sort()
    hash_term = '_'.join(all_terms)
    
    if hash_term in hash_list.keys():
        lst = hash_list[hash_term]
        lst.append(entity)
        hash_list[hash_term] = lst
    else:
        hash_list[hash_term] = [entity]

In [84]:
import datetime

solved = []
proceded = []

for entity, term in synonym[['entity', 'term']].values:
    print(entity)
    start_time = datetime.datetime.now()
    
    cut = synonym[synonym.entity == entity].term.values
    all_terms = np.append(cut, entity)
    
    print(all_terms)
    all_terms.sort()
    hash_term = '_'.join(all_terms)
    
    with pd.HDFStore('neuro_paper_synonym_results.hdf') as hdf:
        if f'/{entity}' in hdf.keys():
            solved.append(entity)
            end_time = datetime.datetime.now()
            print(f'{entity}: {end_time - start_time}')
            print('--------------')
            continue
    
    with nl.scope as e:
    
        for a in all_terms:
            e.FilteredByTerm[e.t, e.s] = (
                e.TermInStudy[e.t, e.s] &
                (e.t == a)
            )

        #(e.VoxelReported @ e.exp(-e.d / 5.0))[e.x1, e.y1, e.z1, e.s] = (
        #    e.FocusReported(e.x2, e.y2, e.z2, e.s)
        #    & e.Voxel(e.x1, e.y1, e.z1)
        #    & (e.d == e.EUCLIDEAN(e.x1, e.y1, e.z1, e.x2, e.y2, e.z2))
        #    & (e.d < 4)
        #)
        
        e.FilteredByNoTerm[e.t, e.s] = (
            ~(nl.exists(e.t1, e.FilteredByTerm[e.t1, e.s] & (e.t1 == e.t))) &
            e.study_det[e.s]
        )

        e.Result[e.x, e.y, e.z, e.fold, e.PROB[e.x, e.y, e.z, e.fold]] = (
           (
               e.VoxelReported[e.x, e.y, e.z, e.s] 
           ) // (
               e.SelectedStudy[e.s] 
               & e.FilteredByTerm[e.t, e.s] 
               & e.StudyFolds[e.s, e.fold]
           )  
        )

        '''e.ResultMean[e.x, e.y, e.z, e.mean(e.PROB)] = (
            e.Result[e.x, e.y, e.z, e.fold, e.PROB]    
        )

        e.ResultStd[e.x, e.y, e.z, e.std(e.PROB)] = (
            e.Result[e.x, e.y, e.z, e.fold, e.PROB]
        )

        e.ResultSummaryStats[
            e.x, e.y, e.z, e.prob_mean, e.prob_std
        ] = (
            e.ResultMean[e.x, e.y, e.z, e.prob_mean] &
            e.ResultStd[e.x, e.y, e.z, e.prob_std]
        )

        e.VoxelActivationImg[agg_create_region_overlay_MNI[e.x, e.y, e.z, e.p]] = (
            e.ResultMean(e.x, e.y, e.z, e.p)
        )'''


        res = nl.query((e.x, e.y, e.z, e.f, e.p), 
                       e.Result[e.x, e.y, e.z, e.f, e.p])
        
        pss = res.as_pandas_dataframe()
        for ent in hash_list[hash_term]:
            pss.to_hdf('neuro_paper_synonym_results.hdf', key=f'{ent}')
        
        end_time = datetime.datetime.now()
        print(f'{entity}: {end_time - start_time}')

fear
['anxiety' 'fear']
fear: 0:00:00.154736
--------------
anxiety
['fear' 'anxiety']
anxiety: 0:00:00.062136
--------------
symptom
['symptoms' 'symptom']
symptom: 0:00:00.075478
--------------
symptoms
['symptom' 'symptoms']
symptoms: 0:00:00.052330
--------------
medication
['treatment' 'therapy' 'dose' 'administration' 'medication']
medication: 0:00:00.041480
--------------
therapy
['treatment' 'medication' 'therapy']
therapy: 0:00:00.033508
--------------
treatment
['medication' 'therapy' 'treatment']
treatment: 0:00:00.043419
--------------
therapy
['treatment' 'medication' 'therapy']
therapy: 0:00:00.039469
--------------
dose
['medication' 'administration' 'dose']
dose: 0:00:00.036312
--------------
administration
['medication' 'dose' 'administration']
administration: 0:00:00.037860
--------------
treatment
['medication' 'therapy' 'treatment']
treatment: 0:00:00.035801
--------------
medication
['treatment' 'therapy' 'dose' 'administration' 'medication']
medication: 0:00:00.03

/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

access: 0:11:04.684150
stroke
['access' 'stroke']
stroke: 0:00:00.076086
--------------
differentiation
['development' 'differentiation']


/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

differentiation: 0:11:14.820928
growing
['development' 'growing']


...

gene: 0:08:36.541247
cognitive performance
['cognitive function' 'cognitive performance']


/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

cognitive performance: 0:08:46.825317
cognitive function
['cognitive performance' 'cognitive function']
cognitive function: 0:00:00.142299
--------------
visual stimulus
['visual stimuli' 'visual stimulus']


/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

visual stimulus: 0:08:49.054292
visual stimuli
['visual stimulus' 'visual stimuli']
visual stimuli: 0:00:00.159050
--------------


In [85]:
hash_list = {}
for entity, term in related[['entity', 'term']].values:
    
    cut = related[related.entity == entity].term.values
    all_terms = np.append(cut, entity)
    all_terms.sort()
    hash_term = '_'.join(all_terms)
    
    if hash_term in hash_list.keys():
        lst = hash_list[hash_term]
        lst.append(entity)
        hash_list[hash_term] = lst
    else:
        hash_list[hash_term] = [entity]

In [87]:
import datetime

solved = []
proceded = []

from tqdm.notebook import trange, tqdm

for entity, term in tqdm(related[['entity', 'term']].values):
    print(entity)
    start_time = datetime.datetime.now()
    
    cut = related[related.entity == entity].term.values
    all_terms = np.append(cut, entity)
    
    print(all_terms)
    all_terms.sort()
    hash_term = '_'.join(all_terms)
    
    with pd.HDFStore('neuro_paper_related_results.hdf') as hdf:
        if f'/{entity}' in hdf.keys():
            solved.append(entity)
            end_time = datetime.datetime.now()
            print(f'{entity}: {end_time - start_time}')
            print('--------------')
            continue
    
    with nl.scope as e:
    
        for a in all_terms:
            e.FilteredByTerm[e.t, e.s] = (
                e.TermInStudy[e.t, e.s] &
                (e.t == a)
            )

        #(e.VoxelReported @ e.exp(-e.d / 5.0))[e.x1, e.y1, e.z1, e.s] = (
        #    e.FocusReported(e.x2, e.y2, e.z2, e.s)
        #    & e.Voxel(e.x1, e.y1, e.z1)
        #    & (e.d == e.EUCLIDEAN(e.x1, e.y1, e.z1, e.x2, e.y2, e.z2))
        #    & (e.d < 4)
        #)
        
        e.FilteredByNoTerm[e.t, e.s] = (
            ~(nl.exists(e.t1, e.FilteredByTerm[e.t1, e.s] & (e.t1 == e.t))) &
            e.study_det[e.s]
        )

        e.Result[e.x, e.y, e.z, e.fold, e.PROB[e.x, e.y, e.z, e.fold]] = (
           (
               e.VoxelReported[e.x, e.y, e.z, e.s] 
           ) // (
               e.SelectedStudy[e.s] 
               & e.FilteredByTerm[e.t, e.s] 
               & e.StudyFolds[e.s, e.fold]
           )  
        )
        
        e.NoResult[e.x, e.y, e.z, e.fold, e.PROB[e.x, e.y, e.z, e.fold]] = (
           (
               e.VoxelReported[e.x, e.y, e.z, e.s] 
           ) // (
               e.SelectedStudy[e.s] 
               & e.FilteredByNoTerm[e.t, e.s] 
               & e.StudyFolds[e.s, e.fold]
           )  
        )

        res = nl.query((e.x, e.y, e.z, e.f, e.p), 
                       e.Result[e.x, e.y, e.z, e.f, e.p])
        
        pss = res.as_pandas_dataframe()
        for ent in hash_list[hash_term]:
            pss.to_hdf('neuro_paper_related_results.hdf', key=f'{ent}')
        
        end_time = datetime.datetime.now()
        print(f'{entity}: {end_time - start_time}')

  0%|          | 0/185 [00:00<?, ?it/s]

epilepsy
['stroke' 'access' 'epilepsy']
epilepsy: 0:00:00.137604
--------------
epilepsy
['stroke' 'access' 'epilepsy']
epilepsy: 0:00:00.057373
--------------
drug
['medication' 'preparation' 'placebo' 'drug']
drug: 0:00:00.048856
--------------
drugs
['medication' 'preparation' 'placebo' 'drugs']
drugs: 0:00:00.060993
--------------
rest
['medication' 'treatment' 'therapy' 'dmn' 'rest']
rest: 0:00:00.053436
--------------
drug
['medication' 'preparation' 'placebo' 'drug']
drug: 0:00:00.049547
--------------
drugs
['medication' 'preparation' 'placebo' 'drugs']
drugs: 0:00:00.072910
--------------
children
['preparation' 'add' 'hyperactivity disorder' 'adhd' 'youth' 'childhood'
 'children']
children: 0:00:00.063310
--------------
drug
['medication' 'preparation' 'placebo' 'drug']
drug: 0:00:00.048401
--------------
drugs
['medication' 'preparation' 'placebo' 'drugs']
drugs: 0:00:00.057011
--------------
eye movement
['eye movements' 'eye movement']
eye movement: 0:00:00.061833
--------

recognition: 0:00:00.051791
--------------
recognition
['vision' 'view' 'hearing' 'cognitive performance' 'cognitive function'
 'recognition']
recognition: 0:00:00.058824
--------------
arterial
['map' 'arterial']
arterial: 0:00:00.051460
--------------
neuron
['rest' 'bold signal' 'selective attention' 'neuron']
neuron: 0:00:00.054306
--------------
neuron
['rest' 'bold signal' 'selective attention' 'neuron']
neuron: 0:00:00.054890
--------------
adaptation
['evolution' 'adaptation']
adaptation: 0:00:00.053048
--------------
medication
['rest' 'drug' 'drugs' 'dot' 'medication']
medication: 0:00:00.049591
--------------
preparation
['drug' 'drugs' 'children' 'preparation']
preparation: 0:00:00.054168
--------------
placebo
['drug' 'drugs' 'placebo']
placebo: 0:00:00.050210
--------------
medication
['rest' 'drug' 'drugs' 'dot' 'medication']
medication: 0:00:00.071714
--------------
preparation
['drug' 'drugs' 'children' 'preparation']
preparation: 0:00:00.092371
--------------
placebo


/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

process: 0:12:26.797764
process
['early stage' 'latency' 'process']
process: 0:00:00.140054
--------------
touch
['communication' 'touch']


...

visual cortex: 0:12:01.985415
anisotropy
['fractional anisotropy' 'anisotropy']
anisotropy: 0:00:00.150647
--------------
tensor
['dti' 'tensor']


/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

tensor: 0:11:53.265201
temporal cortex
['auditory cortex' 'temporal cortex']


/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

temporal cortex: 0:11:52.813165


In [92]:
import datetime

for entity in tqdm(related.entity.unique()):
    print(entity)
    start_time = datetime.datetime.now()

    with pd.HDFStore('neuro_paper_single_terms.hdf') as hdf:
        if f'/{entity}' in hdf.keys():
            end_time = datetime.datetime.now()
            print(f'{entity}: {end_time - start_time}')
            print('--------------')
            continue
    
    with nl.scope as e:
    
        e.FilteredByTerm[e.t, e.s] = (
            e.TermInStudy[e.t, e.s] &
            (e.t == entity)
        )

        #(e.VoxelReported @ e.exp(-e.d / 5.0))[e.x1, e.y1, e.z1, e.s] = (
        #    e.FocusReported(e.x2, e.y2, e.z2, e.s)
        #    & e.Voxel(e.x1, e.y1, e.z1)
        #    & (e.d == e.EUCLIDEAN(e.x1, e.y1, e.z1, e.x2, e.y2, e.z2))
        #    & (e.d < 4)
        #)

        e.Result[e.x, e.y, e.z, e.fold, e.PROB[e.x, e.y, e.z, e.fold]] = (
           (
               e.VoxelReported[e.x, e.y, e.z, e.s] 
           ) // (
               e.SelectedStudy[e.s] 
               & e.FilteredByTerm[e.t, e.s] 
               & e.StudyFolds[e.s, e.fold]
           )  
        )

        '''e.ResultMean[e.x, e.y, e.z, e.mean(e.PROB)] = (
            e.Result[e.x, e.y, e.z, e.fold, e.PROB]    
        )

        e.ResultStd[e.x, e.y, e.z, e.std(e.PROB)] = (
            e.Result[e.x, e.y, e.z, e.fold, e.PROB]
        )

        e.ResultSummaryStats[
            e.x, e.y, e.z, e.prob_mean, e.prob_std
        ] = (
            e.ResultMean[e.x, e.y, e.z, e.prob_mean] &
            e.ResultStd[e.x, e.y, e.z, e.prob_std]
        )

        e.VoxelActivationImg[agg_create_region_overlay_MNI[e.x, e.y, e.z, e.p]] = (
            e.ResultMean(e.x, e.y, e.z, e.p)
        )'''


        res = nl.query((e.x, e.y, e.z, e.f, e.p), 
                       e.Result[e.x, e.y, e.z, e.f, e.p])
        
        pss = res.as_pandas_dataframe()
        pss.to_hdf('neuro_paper_synonym_single_terms.hdf', key=f'{entity}')
        
        end_time = datetime.datetime.now()
        print(f'{entity}: {end_time - start_time}')

  0%|          | 0/95 [00:00<?, ?it/s]

epilepsy


/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

epilepsy: 0:11:13.693236
drug
drug: 0:00:00.171810
--------------
drugs
drugs: 0:00:00.090795
--------------
rest
rest: 0:00:00.083684
--------------
children


/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

...

visual cortex: 0:11:01.736624
anisotropy


/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

anisotropy: 0:10:25.911080
tensor


/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are

tensor: 0:10:44.090052
temporal cortex
temporal cortex: 0:00:00.225454
--------------


In [82]:
import datetime



with nl.scope as e:

    e.FilteredByTerm[e.t, e.s] = (
        e.TermInStudy[e.t, e.s] &
        e.ter[e.t]
    )

    e.Result[e.x, e.y, e.z, e.fold, e.PROB[e.x, e.y, e.z, e.fold]] = (
       (
           e.VoxelReported[e.x, e.y, e.z, e.s] 
       ) // (
           e.SelectedStudy[e.s] 
           & e.FilteredByTerm[e.t, e.s] 
           & e.StudyFolds[e.s, e.fold]
       )  
    )
    
    res = nl.solve_all()

    #pss = res.as_pandas_dataframe()

/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(
/Users/gzanitti/.local/lib/python3.8/site-packages/pandas-1.1.5-py3.8-macosx-10.9-x86_64.egg/pandas/core/reshape/merge.py:1123: UserWarning: You are merging on int and float columns where the float values are